<a href="https://colab.research.google.com/github/alan541/UCC-Dissertation/blob/master/Dissertation(Full_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras


Using TensorFlow backend.


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving DLtrain.csv to DLtrain.csv


In [ ]:
import io
df1 = pd.read_csv(io.BytesIO(uploaded['DLtrain.csv']))


In [ ]:
uploaded = files.upload()

Saving DLtest.csv to DLtest.csv


In [ ]:
df2 = pd.read_csv(io.BytesIO(uploaded['DLtest.csv']))

In [ ]:
dataset1 = df1.values
train_data = dataset1[:,0:41].astype(float)
train_labels = dataset1[:,41]
print(train_data)

[[ 0.  0.  0. ...  1.  0.  0.]
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 0.  0.  0. ...  1.  0.  0.]
 ...
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 0.  0. -1. ...  1.  0.  0.]]


In [ ]:
dataset2 = df2.values
test_data = dataset2[:,0:41].astype(float)
test_labels = dataset2[:,41]
print(test_labels)


[1. 0. 0. ... 1. 0. 1.]


In [ ]:
model = keras.Sequential([
    keras.layers.Dense(128, input_dim = 41, activation='sigmoid'),
    keras.layers.Dense(64, activation= 'sigmoid'),
    keras.layers.Dense(2, activation = 'softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, epochs=100, batch_size= 200)


Epoch 1/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6645 - accuracy: 0.6109
Epoch 2/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6494 - accuracy: 0.6409
Epoch 3/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6476 - accuracy: 0.6417
Epoch 4/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6459 - accuracy: 0.6423
Epoch 5/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6442 - accuracy: 0.6428
Epoch 6/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6434 - accuracy: 0.6426
Epoch 7/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6414 - accuracy: 0.6436
Epoch 8/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6409 - accuracy: 0.6449
Epoch 9/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6392 - accuracy: 0.6454
Epoch 10/100
27/27 [==============================] - 0s 2ms/step - loss: 0.6393 - accuracy: 0.6473
Epoch 11/

In [ ]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

72/72 - 0s - loss: 0.6484 - accuracy: 0.6350

Test accuracy: 0.6349693536758423


In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_data)

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving tenniste.csv to tenniste.csv


In [ ]:
odds = pd.read_csv(io.BytesIO(uploaded['tenniste.csv']))


In [ ]:
datasetodds = odds.values

1.83

In [ ]:
# bet on favourite strategy
bet = 0
win = 0

for i in range(2282):
  if test_labels[i] == 0:
    if predictions[i,0] < 0.50 and datasetodds[i,141] > 0:
      win = win + datasetodds[i,141] 
    if datasetodds[i,141] >0:
      bet = bet + 1
  else:
    if predictions[i,0] >= 0.50 and datasetodds[i,141] > 0:
      win = win + datasetodds[i,141]
    if datasetodds[i,141] >0:
      bet = bet + 1


win = win - bet

ret = win/bet

ret


-0.0564956140350871

In [ ]:
# high probs only betting
bet = 0
win = 0

for i in range(2282):
  if test_labels[i] == 0:
    if predictions[i,0] <= 0.30 and datasetodds[i,141] > 0:
      win = win + datasetodds[i,141] 
      bet = bet + 1
    if predictions[i,0] > 0.70 and datasetodds[i,141] >0:
      bet = bet + 1
  else:
    if predictions[i,0] >= 0.70 and datasetodds[i,141] > 0:
      win = win + datasetodds[i,141]
      bet = bet + 1
    if predictions[i,0] < 0.30 and datasetodds[i,141] >0:
      bet = bet + 1


win = win - bet

ret = win/bet

print(bet)

print(ret)


944
0.001705508474576406


In [ ]:
# bet if expected positive return
bet = 0
win = 0

for i in range(2282):
  if test_labels[i] == 0:
    if predictions[i,0]*datasetodds[i,142] > 1:
      bet = bet + 1
    if (1-predictions[i,0])*datasetodds[i,141] >1:
      bet = bet + 1
      win = win + datasetodds[i,141] 
  else:
    if predictions[i,0]*datasetodds[i,141] > 1:
      bet = bet + 1
      win = win + datasetodds[i,141] 
    if (1-predictions[i,0])*datasetodds[i,142] >1:
      bet = bet + 1
      


win = win - bet

ret = win/bet

ret


2187
-0.035121170553269083


In [ ]:
# bet if model prob - bookie prob > .05
bet = 0
win = 0

for i in range(2282):
  if datasetodds[i,142] > 1:
    if test_labels[i] == 0:
      if predictions[i,0] - (1/datasetodds[i,142]) >= 0.05:
        bet = bet + 1
      if ((1-predictions[i,0]) - (1/datasetodds[i,141])) >= 0.05:
        bet = bet + 1
        win = win + datasetodds[i,141] 
    else:
      if predictions[i,0] - (1/datasetodds[i,141]) >= 0.05:
        bet = bet + 1
        win = win + datasetodds[i,141] 
      if ((1-predictions[i,0]) - (1/datasetodds[i,142])) >= 0.05:
        bet = bet + 1
      

win = win - bet

ret = win/bet

ret


2006
-0.04318045862412824


In [ ]:
# bet on favourite strategy with % of bank roll MM

nb = 0
BR = 2282
bet = 0
win = 0

for i in range(2282):
  stake = 0.01*BR
  if test_labels[i] == 0:
    if predictions[i,0] < 0.50 and datasetodds[i,141] > 0:
      bet = bet + stake
      win = win + (stake*datasetodds[i,141]) 
      BR = BR + ((stake*datasetodds[i,141]) - stake)
      nb = nb + 1
    if  predictions[i,0] > 0.50 and datasetodds[i,141] >0:
      bet = bet + stake
      BR = BR - stake
      nb = nb + 1
  else:
    if predictions[i,0] > 0.50 and datasetodds[i,141] > 0:
      bet = bet + stake
      win = win + (stake*datasetodds[i,141]) 
      BR = BR + ((stake*datasetodds[i,141]) - stake)
      nb = nb + 1
    if  predictions[i,0] < 0.50 and datasetodds[i,141] >0:
      bet = bet + stake
      BR = BR - stake
      nb = nb + 1

    if BR <=0:
      break 


win = win - bet

ret = win/bet

ret


-0.054148961837413495

In [ ]:
# high probs only betting with % of BR MM
bet = 0
win = 0
nb = 0
BR =2282

for i in range(2282):
  stake = .01*BR
  if test_labels[i] == 0:
    if predictions[i,0] <= 0.30 and datasetodds[i,141] > 0:
      bet = bet + stake
      win = win + (stake*datasetodds[i,141])
      BR = BR + ((stake*datasetodds[i,141]) - stake)
      nb = nb + 1
    if predictions[i,0] > 0.70 and datasetodds[i,141] >0:
      bet = bet + stake
      BR = BR - stake
      nb = nb + 1
  else:
    if predictions[i,0] >= 0.70 and datasetodds[i,141] > 0:
      bet = bet + stake
      win = win + (stake*datasetodds[i,141])
      BR = BR + ((stake*datasetodds[i,141]) - stake)
      nb = nb + 1
    if predictions[i,0] < 0.30 and datasetodds[i,141] >0:
      bet = bet + stake
      BR = BR - stake
      nb = nb + 1
  if BR <=0:
      break 

win = win - bet

ret = win/bet

ret


23321.212801318838
-0.015006005758162426


In [ ]:
# bet if expected positive return and % of BR MM
bet = 0
win = 0
BR = 2282
nb = 0

for i in range(2282):
  stake =.01*BR
  if test_labels[i] == 0:
    if predictions[i,0]*datasetodds[i,142] > 1:
      bet = bet + stake
      BR = BR - stake
      nb = nb + 1
    if (1-predictions[i,0])*datasetodds[i,141] >1:
      bet = bet + stake
      win = win + (stake*datasetodds[i,141])
      BR = BR + ((stake*datasetodds[i,141]) - stake)
      nb = nb + 1
  else:
    if predictions[i,0]*datasetodds[i,141] > 1:
      win = win + (stake*datasetodds[i,141])
      bet = bet + stake
      BR = BR + ((stake*datasetodds[i,141]) - stake)
      nb = nb + 1
    if (1-predictions[i,0])*datasetodds[i,142] >1:
      bet = bet + stake
      BR = BR - stake
      nb = nb + 1
  if BR <=0:
      break       


win = win - bet

ret = win/bet

ret


-0.03244021428255258

In [ ]:
# bet if model prob - bookie prob > .05 and % of BR MM
bet = 0
win = 0
nb = 0
BR = 2282

for i in range(2282):
  if datasetodds[i,142] > 1:
    if test_labels[i] == 0:
      if predictions[i,0] - (1/datasetodds[i,142]) >= 0.05:
        bet = bet + stake
        BR = BR - stake
        nb = nb + 1
      if ((1-predictions[i,0]) - (1/datasetodds[i,141])) >= 0.05:
        bet = bet + stake
        win = win + (stake*datasetodds[i,141])
        BR = BR + ((stake*datasetodds[i,141]) - stake)
        nb = nb + 1
    else:
      if predictions[i,0] - (1/datasetodds[i,141]) >= 0.05:
        win = win + (stake*datasetodds[i,141])
        bet = bet + stake
        BR = BR + ((stake*datasetodds[i,141]) - stake)
        nb = nb + 1
      if ((1-predictions[i,0]) - (1/datasetodds[i,142])) >= 0.05:
        bet = bet + stake
        BR = BR - stake
        nb = nb + 1
    if BR <=0:
      break     

win = win - bet

ret = win/bet

ret


-0.043180458624150216

In [ ]:
# bet if expected positive return & KC MM 
bet = 0
win = 0

for i in range(2282):
  stake1 = ((predictions[i,0]*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
  stake3 = ((predictions[i,0]*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
  stake2 = (((1-predictions[i,0])*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
  stake4 = (((1-predictions[i,0])*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
  if test_labels[i] == 0:
    if predictions[i,0]*datasetodds[i,142] > 1:
      bet = bet + stake1
    if (1-predictions[i,0])*datasetodds[i,141] >1:
      bet = bet + stake2
      win = win + (stake2*datasetodds[i,141]) 
  else:
    if predictions[i,0]*datasetodds[i,141] > 1:
      win = win + (stake3*datasetodds[i,141])
      bet = bet + stake3
    if (1-predictions[i,0])*datasetodds[i,142] >1:
      bet = bet + stake4
      


win = win - bet

ret = win/bet

ret


-0.03138208694618706

In [ ]:
# bet if model prob - bookies > .05 & KC MM 
bet = 0
win = 0

for i in range(2282):
  if datasetodds[i,142] > 1:
    stake1 = ((predictions[i,0]*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
    stake3 = ((predictions[i,0]*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
    stake2 = (((1-predictions[i,0])*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
    stake4 = (((1-predictions[i,0])*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
    if test_labels[i] == 0:
      if predictions[i,0] - (1/datasetodds[i,142]) >= 0.05:
        bet = bet + stake1
      if ((1-predictions[i,0]) - (1/datasetodds[i,141])) >= 0.05:
        bet = bet + stake2
        win = win + (stake2*datasetodds[i,141]) 
    else:
      if predictions[i,0] - (1/datasetodds[i,141]) >= 0.05:
        win = win + (stake3*datasetodds[i,141])
        bet = bet + stake3
      if ((1-predictions[i,0]) - (1/datasetodds[i,142])) >= 0.05:
        bet = bet + stake4
        


win = win - bet

ret = win/bet

ret


-0.03193163998006029

In [ ]:
# bet if expected positive return & reverse of KC MM 

bet = 0
win = 0

for i in range(2282):
  stake1 = 1 - ((predictions[i,0]*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
  stake3 = 1 - ((predictions[i,0]*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
  stake2 = 1 - (((1-predictions[i,0])*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
  stake4 = 1 - (((1-predictions[i,0])*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
  if test_labels[i] == 0:
    if predictions[i,0]*datasetodds[i,142] > 1:
      bet = bet + stake1
    if (1-predictions[i,0])*datasetodds[i,141] >1:
      bet = bet + stake2
      win = win + (stake2*datasetodds[i,141]) 
  else:
    if predictions[i,0]*datasetodds[i,141] > 1:
      win = win + (stake3*datasetodds[i,141])
      bet = bet + stake3
    if (1-predictions[i,0])*datasetodds[i,142] >1:
      bet = bet + stake4
      


win = win - bet

ret = win/bet

ret


-0.037941033965749216

In [ ]:
# bet if model prob - bookies > .05 & reverse of KC MM 

bet = 0
win = 0

for i in range(2282):
  if datasetodds[i,142] > 1:
    stake1 = 1 - ((predictions[i,0]*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
    stake3 = 1 - ((predictions[i,0]*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
    stake2 = 1 - (((1-predictions[i,0])*datasetodds[i,141])-1)/(datasetodds[i,141]-1)
    stake4 = 1 - (((1-predictions[i,0])*datasetodds[i,142])-1)/(datasetodds[i,142]-1)
    if test_labels[i] == 0:
      if predictions[i,0] - (1/datasetodds[i,142]) >= 0.05:
        bet = bet + stake1
      if ((1-predictions[i,0]) - (1/datasetodds[i,141])) >= 0.05:
        bet = bet + stake2
        win = win + (stake2*datasetodds[i,141]) 
    else:
      if predictions[i,0] - (1/datasetodds[i,141]) >= 0.05:
        win = win + (stake3*datasetodds[i,141])
        bet = bet + stake3
      if ((1-predictions[i,0]) - (1/datasetodds[i,142])) >= 0.05:
        bet = bet + stake4
        


win = win - bet

ret = win/bet

ret


NameError: ignored